# Sentiment Transfer DNN

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
# load data
from tep.utils import load_array

In [3]:
y = load_array("data/log_labels_v3.bc")
print(y.shape)
X_seq = load_array("data/sequences_len32_v3.bc")
print(X_seq.shape)
X_meta = load_array("data/meta_features_v3.bc")
print(X_meta.shape)

(1293005,)
(1293005, 32)
(1293005, 24)


In [4]:
# load sentiment transfer model
from tep.modelUtils import load_architecture
model = load_architecture("models/tl/baseline.json")
model.load_weights("models/tl/baseline.hdf5")
model.summary()

/Users/felix/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
seq_input (InputLayer)       (None, 32)                0         
_________________________________________________________________
word_emb (Embedding)         (None, 32, 100)           43224100  
_________________________________________________________________
input_dropout (Dropout)      (None, 32, 100)           0         
_________________________________________________________________
padding_1 (ZeroPadding1D)    (None, 34, 100)           0         
_________________________________________________________________
conv_1 (Conv1D)              (None, 32, 128)           38528     
_________________________________________________________________
pool_1 (MaxPooling1D)        (None, 16, 128)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 128)           0         
__________

In [5]:
# compile model
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['acc'])

In [6]:
# test model
preds = model.predict(X_seq[:100])
preds[:10]

array([[0.12762938, 0.87237066],
       [0.07886947, 0.92113054],
       [0.8192652 , 0.18073478],
       [0.12624604, 0.87375396],
       [0.5519293 , 0.44807068],
       [0.98013157, 0.0198684 ],
       [0.3553623 , 0.6446377 ],
       [0.12144363, 0.8785564 ],
       [0.37393486, 0.62606514],
       [0.40144113, 0.59855884]], dtype=float32)

## Define transfer model

In [28]:
from keras.models import Model
from keras.layers import Input, Dropout, Dense, Flatten, Activation
from keras.layers.convolutional import Conv1D, ZeroPadding1D
from keras.layers.pooling import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate

In [29]:
# get relevant weights
emb_weights = model.get_layer(name='word_emb').get_weights()
conv1_weights = model.get_layer(name='conv_1').get_weights()
conv2_weights = model.get_layer(name='conv_2').get_weights()
conv3_weights = model.get_layer(name='conv_3').get_weights()

In [30]:
seq_input = Input(shape=(X_seq.shape[1],), dtype='int32', name='seq_input')
embedding = Embedding(emb_weights[0].shape[0], 
                      emb_weights[0].shape[1], 
                      weights=emb_weights, 
                      input_length=X_seq.shape[1], 
                      trainable=False, 
                      name='word_emb')(seq_input)
x = Dropout(0.2, name='input_dropout')(embedding)
x = ZeroPadding1D(name='padding_1')(x)
x = Conv1D(128, 3, activation='relu', name='conv_1', weights=conv1_weights, trainable=False)(x)
x = MaxPooling1D(name='pool_1')(x)
x = Dropout(0.2, name='dropout_1')(x)
x = ZeroPadding1D(name='padding_2')(x)
x = Conv1D(128, 3, activation='relu', name='conv_2', weights=conv2_weights, trainable=False)(x)
x = MaxPooling1D(name='pool_2')(x)
x = Dropout(0.2, name='dropout_2')(x)
x = ZeroPadding1D(name='padding_3')(x)
x = Conv1D(128, 3, activation='relu', name='conv_3', weights=conv3_weights, trainable=False)(x)
x = MaxPooling1D(name='pool_3')(x)
sent_feats = Flatten(name='flatten')(x)

In [31]:
# get meta inputs
meta_feats = Input(shape=(X_meta.shape[1],), name='meta_input')
norm_feats = BatchNormalization(name='meta_norm')(meta_feats)

In [32]:
concat = concatenate([sent_feats, norm_feats], name='comb_feats')
x = Dense(128, activation='relu', name='fc_1')(concat)
x = Dense(128, activation='relu', name='fc_2')(x)
x = Dense(1, activation='linear', name='raw_output')(x)
output = Activation('relu', name='output')(x)

In [33]:
model = Model(inputs=[seq_input, meta_feats], outputs=[output])
model.compile(optimizer='Adam', loss='mean_squared_error', metrics=[])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
seq_input (InputLayer)          (None, 32)           0                                            
__________________________________________________________________________________________________
word_emb (Embedding)            (None, 32, 100)      43224100    seq_input[0][0]                  
__________________________________________________________________________________________________
input_dropout (Dropout)         (None, 32, 100)      0           word_emb[0][0]                   
__________________________________________________________________________________________________
padding_1 (ZeroPadding1D)       (None, 34, 100)      0           input_dropout[0][0]              
__________________________________________________________________________________________________
conv_1 (Co

## Prepare model training

In [35]:
!mkdir models/tl_sent

In [36]:
cp_file = 'models/tl_sent.hdf5'

In [37]:
from keras.callbacks import ModelCheckpoint, History, EarlyStopping, ReduceLROnPlateau

In [38]:
# create callbacks
cp = ModelCheckpoint(cp_file, save_best_only=True, save_weights_only=True)
hist = History()
es = EarlyStopping(patience=5)
rlr = ReduceLROnPlateau(patience=3)
cbs = [cp, hist, es, rlr]

## Train model

In [39]:
# run if training on full data
Xs_train = X_seq[:-10000]
Xm_train = X_meta[:-10000]
y_train = y[:-10000]
Xs_valid = X_seq[-10000:]
Xm_valid = X_meta[-10000:]
y_valid = y[-10000:]

In [40]:
# run if training on sample
Xs_train = X_seq[:10000]
Xm_train = X_meta[:10000]
y_train = y[:10000]
Xs_valid = X_seq[-10000:]
Xm_valid = X_meta[-10000:]
y_valid = y[-10000:]

In [41]:
model.fit(x={'seq_input': Xs_train, 'meta_input': Xm_train}, 
          y={'output': y_train}, 
          validation_data=({'seq_input': Xs_valid, 'meta_input': Xm_valid}, {'output': y_valid}),
          batch_size=128,
          epochs=100,
          verbose=0,
          shuffle=True,
          callbacks=cbs)

InvalidArgumentError: indices[5,30] = 464249 is not in [0, 432241)
	 [[Node: word_emb_3/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](word_emb_3/embeddings/read, _recv_seq_input_4_0)]]

Caused by op 'word_emb_3/Gather', defined at:
  File "/Users/felix/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/felix/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/felix/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/felix/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/felix/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/felix/anaconda3/lib/python3.6/site-packag

We will abandon this subproject since it conflicts with the constraint of not requiring additional data for model training.